In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://pbs.twimg.com/media/E6EFuIPXEAA9IXP.jpg)twitter.com

In [ ]:
train = pd.read_csv('../input/sentiment-dataset-for-afghanistan-sd4a/SD4A-clean.csv')
train.head().style.set_properties(**{'background-color':'black',
                                     'color': '#03e8fc'})

In [ ]:
train.isnull().sum()

In [ ]:
train = train.rename(columns={'Wed Jun 13 06:41:31 +0000 2018':'date', '1': 'polarity', 'assure afghans of bright future once u.s invaders by reuters news': 'text'})

In [ ]:
train.head()

#Code by James McNeill  https://www.kaggle.com/datajmcn/nlp-read-eda/notebook

In [ ]:
# Extract insights from the excerpt variable
import spacy

In [ ]:
# Initialise spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Perform initial test on one text
sample = train.head(5)
sample
sample1 = train.loc[0, 'text']

In [ ]:
# Create the spacy doc item for review
doc = nlp(sample1)
doc

In [ ]:
# Reviewing the token, lemma and stopword for each token (item)
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma', 'Stopword'))
print("-"*40)
# Review the first 20 values to test the output
for token in doc[:20]:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}\t\t{len(token)}")

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Review stop words</span></h1><br>

In [ ]:
# A few different options for stopwords, spacy and nltk. Lets compare
import nltk
from nltk.corpus import stopwords

In [ ]:
# Comparison of the stop words available
print(f"NLTK : {len(stopwords.words('english'))} \n {stopwords.words('english')}")
print(f"Spacy : {len(nlp.Defaults.stop_words)} \n {nlp.Defaults.stop_words}")

# Compare the differences
nltk_set = set(stopwords.words('english'))
spacy_set = set(nlp.Defaults.stop_words)

# Union - all values
union = nltk_set.union(spacy_set)
# Intersection - seen in both sets
inter = nltk_set.intersection(spacy_set)
print(f"Seen in both : {len(inter)} \n {inter}")
# Remainder - differences between sets
nltk_extra = nltk_set - inter
spacy_extra = spacy_set - inter
print(f"Extra NLTK : {len(nltk_extra)} \n {nltk_extra}")
print(f"Extra Spacy : {len(spacy_extra)} \n {spacy_extra}")

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Review Tfidftransformer and Tfidfvectorizer</span></h1><br>

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#instantiate CountVectorizer() 
cv=CountVectorizer() 

# this steps generates word counts for the words in the sample doc
word_count_vector=cv.fit_transform(sample.text)

word_count_vector.shape

In [ ]:
# Compute the IDF values
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

df_idf.describe()

#Lower the IDF value the more common the value is

In [ ]:
# Time to compute the TFIDF
# count matrix 
count_vector=cv.transform(sample.text) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [ ]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Tfidfvectorizer Usage</span></h1><br>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(sample.text)

In [ ]:
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
 
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here. Here is Sample
fitted_vectorizer=tfidf_vectorizer.fit(sample.text)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(sample.text)
df = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=fitted_vectorizer.get_feature_names(), columns=sample['date'])
df.head()
df.columns
df.shape
df_out = df.loc[:, ['Wed Jun 13 06:42:06 +0000 2018']]
df_out.sort_values(by=['Wed Jun 13 06:42:06 +0000 2018'], ascending=False)

In [ ]:
# Test on all training data (=df)
# just send in all your docs here
fitted_vectorizer=tfidf_vectorizer.fit(train.text)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(train.text)
df = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=fitted_vectorizer.get_feature_names(), columns=train['date'])
df.head()
df.columns
df.shape
df_out = df.loc[:, ['Wed Jun 13 06:42:06 +0000 2018']]
df_out.sort_values(by=['Wed Jun 13 06:42:06 +0000 2018'], ascending=False)

In [ ]:
# Lets create a dictionary to review the key phrase outputs
from collections import defaultdict, Counter

# Returns integers that map to parts of speech
counts_dict = doc.count_by(spacy.attrs.IDS['POS'])

# Print the human readable part of speech tags
for pos, count in counts_dict.items():
    human_readable_tag = doc.vocab[pos].text
    print(human_readable_tag, count)

In [ ]:
pos_counts = defaultdict(Counter)
for token in doc:
    pos_counts[token.pos][token.orth] += 1
    
for pos_id, counts in sorted(pos_counts.items()):
    pos = doc.vocab.strings[pos_id]
    for orth_id, count in counts.most_common():
        print(pos, count, doc.vocab.strings[orth_id], len(doc.vocab.strings[orth_id]))

#SPACE value appears to correspond to the new line.

In [ ]:
# Expanding named entities
for entity in doc.ents:
    print(entity.text, entity.label_)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
print("Number of sentences", len([*doc.sents]))
print("Sentiment", doc.sentiment)

# Understand the length of sentences
for sent in doc.sents:
    print(sent.start_char, sent.end_char, (sent.end_char - sent.start_char))

In [ ]:
from spacy import displacy

# Display the entities within a sentence
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# Visualise the dependencies within a sentence
displacy.render(doc, style='dep', jupyter=True)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Create the datasets for training the models</span></h1><br>

#That below will take some time.

In [ ]:
# Lets apply the nlp instance to each text
train['text_scy'] = train['text'].apply(nlp)

In [ ]:
# Check the data type for the updated column
type(train.loc[0, 'text_scy'])
train.head()

In [ ]:
# Reviewing a different row
train.loc[1,'text_scy']

In [ ]:
# Create the class methods required to run the analysis
class NLPMethods():
    # Create constructor for the class
#     def __init__():
    
    # Number of sentences
    def number_sentences(self, nlp_text):
        return len([*nlp_text.sents])
    
    # Average length of sentence
    def average_sentence_length(self, nlp_text):
        sent_length = list()
        for sent in nlp_text.sents:
            sent_length.append(sent.end_char - sent.start_char)
        return np.mean(sent_length)
    
    # Part of speech tags
    def part_of_speech_tags(self, nlp_text):
        counts_dict = nlp_text.count_by(spacy.attrs.IDS['POS'])
        counts_dict1 = {}
        # Extract the text that matches to the POS value
        for k, v in counts_dict.items():
            counts_dict1[nlp_text.vocab[k].text] = v
        return counts_dict1
    
    # Number of spaces
    def number_spaces(self, nlp_text):
        dict_pos = self.part_of_speech_tags(nlp_text)
        if dict_pos.get('SPACE') != None:
            space = dict_pos.get('SPACE')
        else:
            space = 0
        return space
    
    # Part of speech tags - including the word counts
    def word_counts(self, nlp_text):
        pos_counts = defaultdict(Counter)
        for token in nlp_text:
            pos_counts[token.pos][token.orth] += 1
        
        # Create dictionary for the word counts
        word_counts_dict = {}
        for pos_id, counts in sorted(pos_counts.items()):
            pos = nlp_text.vocab.strings[pos_id]
            for orth_id, count in counts.most_common():
                word_counts_dict[nlp_text.vocab.strings[orth_id]] = {'count':count, 
                                                                     'length':len(nlp_text.vocab.strings[orth_id]), 
                                                                     'pos':pos}
        return word_counts_dict
    
    # Number of words
    def number_words(self, nlp_text):
        dict_word_counts = self.word_counts(nlp_text)
        return len(dict_word_counts.items())
    
    # Longest word
    def longest_word(self, nlp_text):
        dict_word_counts = self.word_counts(nlp_text)
        df = pd.DataFrame(dict_word_counts).T.reset_index().rename(columns={'index':'variable'})
        return max(df['length'])

#Another one snippet that takes time

In [ ]:
# Add columns for the spacy doc
train['num_sentences'] = train['text_scy'].apply(NLPMethods().number_sentences)
train['avg_sentence_length'] = train['text_scy'].apply(NLPMethods().average_sentence_length)
train['pos_dict'] = train['text_scy'].apply(NLPMethods().part_of_speech_tags)
train['num_space'] = train['text_scy'].apply(NLPMethods().number_spaces)
train['wc_dict'] = train['text_scy'].apply(NLPMethods().word_counts)
train['num_words'] = train['text_scy'].apply(NLPMethods().number_words)
train['longest_word'] = train['text_scy'].apply(NLPMethods().longest_word)

In [ ]:
train.sample(5)

In [ ]:
# Review the max value target variable. We don't target to return a single line max val
#max_val = np.max(train['polarity'])
#train_max = train.loc[(train['polarity']==max_val), :]
#train_max

In [ ]:
# Check the reason for the largest target value
#We don't have largest target. Hence, I kept the same value 2829 
type(train.loc[2829, 'text_scy'])
train.loc[2829, 'text_scy']
train.loc[2829, 'text']

In [ ]:
#I commented since we don't have target or min value target.
# Review the min value target variable
#min_val = np.min(train['target'])
#train_min = train.loc[(train['target']==min_val), :]
#train_min

In [ ]:
#Though we don't have smallest target I kept the same value
# Check the reason for the smallest target value
type(train.loc[1705, 'text_scy'])
train.loc[1705, 'text_scy']

#Thanks James McNeill for the code https://www.kaggle.com/datajmcn/nlp-read-eda/notebook